### -------------------- SLOW CODE -----------------------
--------- IMPORT EVERYTHING , TRY MANY ATTEMPET TILL SUCCES ---------

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Enhanced configuration
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
]

HEADERS = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'DNT': '1',
    'Referer': 'https://jadaero.com/',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Upgrade-Insecure-Requests': '1'
}

# Configure session with aggressive retry
session = requests.Session()
retry_strategy = Retry(
    total=10,
    backoff_factor=2,
    status_forcelist=[429, 500, 502, 503, 504, 520, 521, 522, 523, 524],
    allowed_methods=["GET", "HEAD"]
)
adapter = HTTPAdapter(max_retries=retry_strategy, pool_connections=50, pool_maxsize=50)
session.mount("https://", adapter)
session.mount("http://", adapter)

def make_request(url):
    """Make request with randomized headers and delays"""
    headers = HEADERS.copy()
    headers['User-Agent'] = random.choice(USER_AGENTS)
    
    try:
        time.sleep(random.uniform(10, 30))  # 10-30 second delay
        
        response = session.get(
            url,
            headers=headers,
            timeout=(15, 45),  # 15s connect, 45s read timeout
            allow_redirects=True
        )
        
        # Additional verification
        if response.status_code == 200 and "wcpt-table" in response.text:
            return response
        else:
            raise requests.exceptions.RequestException(f"Unexpected response: {response.status_code}")
            
    except Exception as e:
        print(f"Request failed: {str(e)[:200]}")
        raise

def scrape_page(page_num):
    url = f'https://jadaero.com/aircraft-parts-inventory/?24951_paged={page_num}'
    
    for attempt in range(5):  # Try each page up to 5 times
        try:
            response = make_request(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table', {'class': 'wcpt-table'})
            
            if not table:
                raise ValueError("Table not found in page")
                
            rows = []
            for row in table.find_all('tr')[1:]:
                try:
                    cells = row.find_all('td')
                    if len(cells) >= 5:
                        img_tag = cells[0].find('img')
                        name_tag = cells[1].find('a')
                        
                        rows.append([
                            description.split('Part Number:')[1].split('\n')[0].strip() if 'Part Number:' in (description := cells[2].get_text(separator='\n').strip()) else '',
                            name_tag.text.strip() if name_tag else '',
                            description,
                            cells[3].find('input', {'name': 'quantity'})['value'] if cells[3].find('input', {'name': 'quantity'}) else '1',
                            img_tag['src'] if img_tag else '',
                            name_tag['href'] if name_tag else ''
                        ])
                except Exception as e:
                    print(f"Error processing row: {str(e)[:200]}")
                    continue
            
            return rows
            
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for page {page_num}: {str(e)[:200]}")
            time.sleep(60 * (attempt + 1))  # Wait 1-5 minutes between attempts
            continue
    
    return None  # All attempts failed

# Main scraping process
with open('jadaero_parts.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Part Number', 'Name', 'Description', 'Quantity', 'Image URL', 'Product URL'])
    
    for page in range(1, 51):
        print(f"\nProcessing page {page}...")
        
        data = scrape_page(page)
        if data:
            writer.writerows(data)
            print(f"Successfully saved {len(data)} items from page {page}")
        else:
            print(f"Failed to process page {page} after multiple attempts")
            
        # Random long delay between pages
        time.sleep(random.uniform(30, 120))

print("\nScraping completed! Results saved to jadaero_parts.csv")


Processing page 1...
Request failed: Unexpected response: 403
Attempt 1 failed for page 1: Unexpected response: 403
Request failed: Unexpected response: 403
Attempt 2 failed for page 1: Unexpected response: 403
Request failed: Unexpected response: 403
Attempt 3 failed for page 1: Unexpected response: 403
Successfully saved 10 items from page 1

Processing page 2...
Successfully saved 10 items from page 2

Processing page 3...
Request failed: Unexpected response: 403
Attempt 1 failed for page 3: Unexpected response: 403
Successfully saved 10 items from page 3


### -------------------- FAST CODE ------------------- 
--------- SKIP NON IMPORTANT OR CAUSE OF CRASH PAGE ---------

In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
import os  # Added this missing import

# Lightweight configuration
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
]

def scrape_page(page_num):
    headers = {
        'User-Agent': random.choice(USER_AGENTS),
        'Accept-Language': 'en-US,en;q=0.9'
    }
    
    url = f'https://jadaero.com/aircraft-parts-inventory/?24951_paged={page_num}'
    
    try:
        # Random delay between 5-8 seconds
        time.sleep(random.uniform(5, 8))
        
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            return None
            
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'class': 'wcpt-table'})
        if not table:
            return None
            
        rows = []
        for row in table.find_all('tr')[1:]:
            try:
                cells = row.find_all('td')
                if len(cells) >= 5:
                    rows.append([
                        cells[1].find('a').text.strip() if cells[1].find('a') else '',
                        cells[2].get_text(separator=' ').strip(),
                        cells[3].find('input', {'name': 'quantity'})['value'] if cells[3].find('input') else '1'
                    ])
            except:
                continue
                
        return rows
        
    except:
        return None

# Main scraping process
with open('jadaero_parts_fast.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Part Name', 'Description', 'Quantity'])
    
    for page in range(1, 51):
        print(f"Page {page}", end='\r')
        
        data = scrape_page(page)
        if data:
            writer.writerows(data)  # Fixed typo here (was writer.writerows)
            print(f"Page {page}: {len(data)} items")
        else:
            print(f"Failed page {page} - will retry later")
            # Add failed pages to retry queue
            with open('retry_pages.txt', 'a') as retry_file:
                retry_file.write(f"{page}\n")
        
        # Small random delay even on failures
        time.sleep(random.uniform(3, 5))

print("\nInitial scrape complete! Now retrying failed pages...")

# Retry failed pages (only if needed)
if os.path.exists('retry_pages.txt'):
    with open('retry_pages.txt', 'r') as f:
        pages_to_retry = [int(line.strip()) for line in f]
    
    for page in pages_to_retry:
        print(f"Retrying page {page}", end='\r')
        data = scrape_page(page)
        if data:
            with open('jadaero_parts_fast.csv', 'a', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerows(data)
            print(f"Successfully retried page {page}")
        
        time.sleep(8)  # Longer delay for retries

    # Clean up the retry file
    os.remove('retry_pages.txt')

print("\nAll done! Results saved to jadaero_parts_fast.csv")

Page 1: 10 items
Page 2: 10 items
Page 3: 10 items
Page 4: 10 items
Page 5: 10 items
Page 6: 10 items
Page 7: 10 items
Page 8: 10 items
Page 9: 10 items
Page 10: 10 items
Page 11: 10 items
Page 12: 10 items
Page 13: 10 items
Page 14: 10 items
Page 15: 10 items
Page 16: 10 items
Page 17: 10 items
Page 18: 10 items
Page 19: 10 items
Page 20: 10 items
Page 21: 10 items
Page 22: 10 items
Page 23: 10 items
Page 24: 10 items
Page 25: 10 items
Page 26: 10 items
Page 27: 10 items
Page 28: 10 items


KeyboardInterrupt: 